In [1]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import glob
# from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from itertools import chain #untuk flaten array 2d
from collections import defaultdict
from collections import OrderedDict


# Preprocessing 

In [2]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

### normalisasi

In [3]:
def normalisasi(body):
    
    hasil = []
    temp_str = ''

    #hapus tag
    hapusTag = r'<DATE>|</DATE>|<TITLE>|</TITLE>|<BODY>|</BODY>'
    body = re.sub(hapusTag, ' ', body)
    
    #hapus \n
    hapusEnter = r'\n'
    body = re.sub(hapusEnter, ' ', body)
    
    #hapus spasi lebih dari satu
    spasiLebihDariSatu = r'\s+'
    body = re.sub(spasiLebihDariSatu, ' ', body)
    
    # hapus htmlEntities 
    htmlEntities = r"&lt;"
    body = re.sub(htmlEntities, '', body)
    
    #ambil tanggal
    tanggal = r"((\d{2}|\d{4})[.-](J(anuary|an|AN|une|un|UN|uly|ul|UL)|F(ebruary|eb|EB)|M(arch|ar|AR|ay|AY)|A(pril|pr|PR|ugust|ug|UG)|S(eptember|ep|EP)|O(ctober|ct|CT)|N(ovember|ov|OV)|D(ecember|ec|EC))[-.](\d{4}|\d{2}))"
    list_tanggal = re.findall(tanggal,body)
    hasil.append(list_tanggal)
    body = re.sub(tanggal, "", body)
    
    #ambil waktu
    waktu = r"(\d{2}:\d{2}:\d{2}(\.)?\d{2})"
    list_waktu = re.findall(waktu,body)
    hasil.append(list_waktu)
    body = re.sub(waktu, "", body)
    
    #ambil angka desimal, persen, mata uang 
    desimal = "(([+-]?(\$|£)?(\d{1,3}[.,])*(\d+)(%)?))"
    list_desimal = re.findall(desimal, body)
    hasil.append(list_tanggal)
    body = re.sub(desimal, "", body)
    
    #email
    email = "([\w-]+(\.[\w-]+|\.)*@[\w-]+(\.[\w-]+)+)"
    list_email = re.findall(email,body)
    hasil.append(list_email)
    body = re.sub(email, "", body)
    
    #url
    url = "((http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?)"
    list_url = re.findall(url,body)
    hasil.append(list_url)
    body = re.sub(url, "", body)
    
    tanda_baca = r'["()<>+?.\[\]{}:,\'\'/-]'
    body = re.sub(tanda_baca, ' ', body)

    body = re.sub(spasiLebihDariSatu, ' ', body)
    
    kutips = r"'s"
    body = re.sub(kutips, '', body)
    
    list_flat = list(chain.from_iterable(hasil))
    
    for i in list_flat:
        temp_str = temp_str + i[0] + " "
    
    body = body + " " + temp_str
    
    body = re.sub(spasiLebihDariSatu, ' ', body)
    
    return body

In [5]:
def readText(file):
    words = []
    f = open(file, 'r') #open file
    text = f.read()    
    f.close()

    return text

In [8]:
def removeEnter(listOfWords): #['<BODY>\n']
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].replace('\n','')
        
    return listOfWords

In [4]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [6]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

## Load Stop Word from NLTK

In [7]:
stop_words = stopwords.words('english')

In [9]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

### Load Lemmatizer from NLTK

In [11]:
lemmatizer = WordNetLemmatizer()

In [10]:
def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

### Load Stemming from NLTK

In [12]:
def stemming(listOfWords):
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in listOfWords]
    
    return stemmed

### Token

In [15]:
def make_token(txt):
    list_hasil = txt.split(" ")            
    return list_hasil

### Term

In [14]:
def make_term(file_name_token,file_name_term):
    list_term = []
    f = open(file_name_token, 'r')
    text = f.read()
    f.close()
    
    list_token = text.split("\n")
    for i in list_token:
        if i not in list_term:
            list_term.append(i)
            
    f1 = open(file_name_term,'a+')
    # text = concate_term_newline(list_term)
#     print(text)    
    f1.write(list_term)
    f1.close()


In [13]:
def concate_term_newline(dist_in):
    hasil = ''
    str_temp2 = ''
    is_first = True
    for i in dist_in:
        for j in dist_in[i]:
            if is_first:
                str_temp2 = j
                is_first = False
            else:
                str_temp2 = str_temp2 + ","+ j
        is_first = True
        str_temp = i+ "\t" + str_temp2+ "\n"
        str_temp2 = ""
        hasil = hasil + str_temp
        
    return hasil

### tulis data

In [16]:
def writeFile(dict_in,file_name):
    
    str_of_list = concate_term_newline(dict_in)
    
#     print(str_of_list)
    saveFile = open(file_name, 'w+' )
    saveFile.write(str_of_list)
    saveFile.close()
#     print(list_hasil)
    

### Add Doc Number

In [17]:
def addDocNumber(_listIn, _docNum):
#     _docNum[14:18]
    for i in range(len(_listIn)):
        _listIn[i] = _listIn[i] + '_' + _docNum[14:18]
        
    return _listIn

## Preprocessing Main

In [18]:
listOfFiles = readFile('DataRouter')
listOfWordsOfFile = []
counter = 1
blok = 10

dict_list_term = defaultdict(list)
dict_list_term2 = defaultdict(list)
blok_file_name = "term_blok"
blok_name = []

for i in listOfFiles: #iterasi tiap file
    txt = readText(i)
    
    txt = normalisasi(txt)
    listOfWords = make_token(txt)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = stemming(listOfWords)
    listOfWords = lemmatization(listOfWords)
    
#     Diganti KEVIN
#     listOfWords = addDocNumber(listOfWords,i)
    
    for j in listOfWords:
#         if j not in list_term:
#         list_term[j].append(i) #15:01:01.79 : ,DataRouter\Doc0001.txt
        dict_list_term2[j]
        dict_list_term[j].append(i[14:18]) #15:01:01.79 : 0001

#sorting ascending
dict_list_term = OrderedDict(sorted(dict_list_term.items()))


### Tulis Hasil Preprocessing

In [19]:
writeFile(dict_list_term2,"hasil_inverted.txt")
writeFile(dict_list_term,"hasil_inverted_no_doc.txt")


# Lompatan kuery 

##  Wild Card

### Bigram

In [20]:
def generateBigram(_queri):
    hasil = []
    for i in range(len(_queri)-1):
        if i == 0:
            hasil.append('$'+_queri[i])
        else:
            hasil.append(_queri[i] + _queri[i+1])
            
    return hasil

In [21]:
def wildCard(_queri):
    queri = _queri.replace('*','')
    bigram = generateBigram(queri)
    
    return bigram

## Koreksi Ejaan

### Kata Terisolasi

In [22]:
#from nltk.corpus import words #kamus inggris
from nltk.metrics import * #hitung jarak

In [23]:
def spellingCorection(_word):
    wordsSelected = identicalWord(_word)
    
    thresholdDistance = 3 #threshold
    mostIdenticalWord = []
    for i in wordsSelected: #check dengan levenshtaien
        tempDistance = edit_distance(i, _word)
        if tempDistance < thresholdDistance:
            sortestDistance = tempDistance
            mostIdenticalWord.append(i)
            
    return mostIdenticalWord

In [24]:
def identicalWord(_word):
    wordLenght = len(_word)
    wordsSelected = [] #word yang selisih panjang = 0
    
#     listOfWord = getListOfWord('hasil_inverted.txt')
    
    for word in listOfWordOfInvIndex: #tadinya listOfWord (yg di atas)
        if abs(len(word) - wordLenght) == 0 and word[0] == _word[0]: #panjangnya sama dan huruf depan sama
            wordsSelected.append(word)
            
    return wordsSelected

In [25]:
def getListOfWord(_path):
    f = open(_path,"r")
    words = f.read()
    words = words.replace('\t','')
    listOfWord = words.split('\n')
    
    return listOfWord

## Pre-Procesing Queri

In [26]:
from nltk.stem import WordNetLemmatizer

In [27]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        if listOfWords[i] != 'OR' and listOfWords[i] != 'AND' and listOfWords[i] != 'NOT':
            listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

In [28]:
lemmatizer = WordNetLemmatizer()

def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

In [29]:
def tokenize(_queri):
    listOfWord = _queri.split(' ')
    
    return listOfWord

In [30]:
from nltk.stem.snowball import SnowballStemmer

def stemming(listOfWords):
    stemmer = SnowballStemmer("english")
#     stemmed = [stemmer.stem(word) for word in listOfWords]
    for i in range(len(listOfWords)):
        if listOfWords[i] != 'OR' and listOfWords[i] != 'AND' and listOfWords[i] != 'NOT':
            listOfWords[i] = stemmer.stem(listOfWords[i])
    
    return listOfWords

In [31]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

In [32]:
def preProcessingQuery(_query):
    listOfWords = tokenize(_query)
#     print('token : ',listOfWords)
    listOfWords = caseFolding(listOfWords)
#     print('case : ',listOfWords)
    listOfWords = removeStopWord(listOfWords)
#     print('remove : ',listOfWords)
    listOfWords = stemming(listOfWords)
#     print('stem : ',listOfWords)
    listOfWords = lemmatization(listOfWords)
#     print('lemma : ',listOfWords)
    
    return listOfWords

# Koreksi kuery Main

In [33]:
def findBoolean(_listOfWord, _bool):
    boolIndex = []
#     listOfWord = _query.split(' ')
    for i in range(len(_listOfWord)):
        if _listOfWord[i] == _bool:
            boolIndex.append(i)
            
    return boolIndex

In [34]:
def isWordInTerm(_listOfWord):
    newListOfWord = []
    temp = ''
    for i in _listOfWord:
        if i != 'OR' and i != 'AND' and i != 'NOT':
            if i not in listOfWordOfInvIndex:
#                 print(i)
                try:
                    temp = spellingCorection(i)[0]
                    newListOfWord.append(temp)
                except:
                    print("No '",i,"' in Term")
                    newListOfWord.append(temp)
            else:
                newListOfWord.append(i)
        else:
            newListOfWord.append(i)
                
    return newListOfWord

In [35]:
def queryProcessing(_query):
    andWord = []
    orWord = []
    notWord = []
    
    tempListOfWord = preProcessingQuery(_query) #pre processing query
#     print('pre : ', tempListOfWord)
    tempListOfWord = isWordInTerm(tempListOfWord) #koreksi query 
#     print('isword : ', tempListOfWord)
    listOfWord = []
    listOfWord = [tempListOfWord[0]]
    
    for i in range(1,len(tempListOfWord)): #mengubah NOT menjadi ~
        if tempListOfWord[i] == 'NOT':
            listOfWord.append('~' + tempListOfWord[i+1])
        elif tempListOfWord[i-1] != 'AND' and tempListOfWord[i] != 'AND' and tempListOfWord[i-1] != 'NOT':
            listOfWord.append('OR')
            listOfWord.append(tempListOfWord[i])
        elif tempListOfWord[i-1] != 'NOT':
            listOfWord.append(tempListOfWord[i])
        
            
    if '*' in _query:
        query = wildCard(_query)
    else:
#         if 'AND' in _query:
#             andIndex = findBoolean(listOfWord, 'AND')
            
#             for i in andIndex:
#                 andWord.append(listOfWord[i-1])
#                 andWord.append(listOfWord[i+1])
                    
#         if 'NOT' in _query:
#             for i in listOfWord:
#                 if i[0] == '~':
#                     notWord.append(i)
                    
        if 'AND' not in _query and 'OR' not in _query:
            orWord = listOfWord
#         elif 'OR' in _query:
#             orIndex = findBoolean(listOfWord, 'OR')
#             for i in orIndex:
#                 orWord.append(listOfWord[i-1])
#                 orWord.append(listOfWord[i+1])
#         else:
#             for i in range(0,len(listOfWord)-1):
#                 if listOfWord[i+1] != 'AND' and listOfWord[i] != 'AND':
#                     orWord.append(listOfWord[i])
#                     orWord.append(listOfWord[i+1])
                
#         print('OR : ',orWord)
#         print('AND : ',andWord)
#         print('NOT : ',notWord)
    return listOfWord 

In [36]:
listOfWordOfInvIndex = getListOfWord('hasil_inverted.txt')
queryProcessing('makan minum AND mandi')

['manag', 'OR', 'minim', 'AND', 'manag']

# Lompatan Pointer

In [37]:
# hanya untuk operator and
def lompatan_pointer(docId_p1,docId_p2):
    is_null_p1 = True
    is_null_p2 = True
    p1 = 0
    p2 = 0
    
    answer = []
    lompatan = 4

# lompatan_pointer
    while is_null_p1 and is_null_p2:
        # jika salah satu list term kosong maka returnnya langsung kosong
        if len(docId_p2)== 0 or len(docId_p1)== 0:
            return answer
        
        if docId_p1[p1] == docId_p2[p2]:
            answer.append(docId_p1[p1])
            p1 = p1+1
            p2 = p2+1 
        elif docId_p1[p1] < docId_p2[p2]:
            if (p1%lompatan) == 0 and docId_p1[p1+lompatan]<= docId_p2[p2]:
                while (p1%lompatan) == 0 and docId_p1[p1+lompatan] <= docId_p2[p2]:
                    p1 = p1 + lompatan
                    if (len(docId_p1)-lompatan) <= p1:
                        is_null_p1 = False
                        break
            else:
                p1 = p1 + 1
                
        elif (p2%lompatan) == 0 and docId_p2[p2+lompatan]<= docId_p1[p1]:
            while (p2%lompatan) == 0 and docId_p2[p2+lompatan] <= docId_p1[p1]:
                p2 = p2 + lompatan
                if (len(docId_p2)-lompatan) <= p2:
                    is_null_p2 = False
                    break
        else:
            p2 = p2 + 1
        
        if (len(docId_p1)-lompatan) <= p1:
            is_null_p1 = False
            
        if (len(docId_p2)-lompatan) <= p2:
            is_null_p2 = False
        
    return answer

In [38]:
# untuk operasi or 
def operasi_or(docId_p1,docId_p2):
    hasil = docId_p1+docId_p2
    return hasil


## Membuat list untuk semua nomor dokumen

In [39]:
list_all_doc_no = []
for i in range(1,501):
    pjg_digit = 4 - len(str(i))
    temp_str = ('0' * pjg_digit) + str(i)
    list_all_doc_no.append(temp_str)
# list_all_doc_no

### Mengambil dokumen id untuk sebuah term

In [40]:
def get_doc_id(t):
    docId_p = []
    if t in dict_list_term: #jika term yang ingin di cari tidak ada pada dict_list_term
        if t[0] == "~":
            t = t.replace("~", "")
            temp_doc_id = dict_list_term[t] 
            docId_p = [ elem for elem in list_all_doc_no if elem not in temp_doc_id]
        else:
            docId_p = dict_list_term[t]
    
    return docId_p

In [41]:
def main2(t1_in,t2_in,oper, cari_term):
    hasil = []
    t1 = []
    t2 = []
    
    if cari_term == True :
        t1 = get_doc_id(t1_in)
    else:
        t1 = t1_in
        
    t2 = get_doc_id(t2_in)
    
    if oper == "AND":
        hasil = lompatan_pointer(t1,t2)
    elif oper == "OR":
        hasil = operasi_or(t1,t2)
    
    return hasil

In [42]:
def main3(case):
    operator = case[1:len(case):2]
    term = case[0:len(case):2]

    hasil = main2(term[0],term[1],operator[0],True)


    for i in range(1,len(operator)):
        hasil = main2(hasil, term[i+1], operator[i],False)
        
    return hasil

In [43]:
# main3(case3)

# Main Lompatan Pointer 

In [44]:
def main(case):
    # mencari posisi operator "AND" kemudian di selesaikan dulu baru digabungkan dengan semua operator "OR"
    get_index = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
    and_index = get_index("AND",case)
    case_temp = case[:]
    hasil = []
    p2 = 0
    p = 0
    hasil_and = []
    
    if len(case) == 1:
        if case[0] in dict_list_term:
            hasil = dict_list_term[case[0]]
            return hasil
        else:
            print("Key tidak terdapat dalam list term")
            return
    
    while(p < len(and_index)-1):
        if (and_index[p+1] - and_index[p]) != 2:
    #         temp1 = str("gabung: " + str(and_index[p]-1) + " dan " + str(and_index[p]+1))
            t1 = case[and_index[p]-1]
            t2 = case[and_index[p]+1]
            temp1 = main2(t1,t2,"AND",True)
            hasil_and.append(temp1)
            case_temp[and_index[p]-1] = p2
            case_temp[and_index[p]] = p2
            case_temp[and_index[p]+1] = p2
            
            p2 +=1
            p = p+1
            
            
            
        else: 
            temp = []

            while p < len(and_index)-1:
                if (and_index[p+1] - and_index[p]) == 2:
                    temp.append(and_index[p])
                else:
                    break
                p = p + 1

            if and_index[p] - and_index[p-1] == 2:
                temp.append(and_index[p])
                p += 1
            list_and = case[temp[0]-1:temp[-1]+2]
            
            for i in range(temp[0]-1,temp[-1]+2):
                case_temp[i] = p2
            
            temp_hasil = main3(list_and)
            hasil_and.append(temp_hasil)
            p2 +=1

        # karena panjangnya cuma sampai n, sehingga untuk term yang operasi paling terakhir di lakukan setelah loop 
        if p == len(and_index)-1:
#             temp1 = str("gabung: " + str(and_index[p]-1) + " dan " + str(and_index[p]+1))
            t1 = case[and_index[p]-1]
            t2 = case[and_index[p]+1]
            temp1 = main2(t1,t2,"AND",True)
            hasil_and.append(temp1)
            case_temp[and_index[p]-1] = p2
            case_temp[and_index[p]] = p2
            case_temp[and_index[p]+1] = p2
            p2 +=1
            p = p+1
    
    #-----------------------------------=---------------------------
    #gabung dengan or
    
    get_index = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
    or_index = get_index("OR",case_temp)
    
    for i in range(len(or_index)):
        t1 = []
        t2 = []
        if isinstance(case_temp[or_index[i]-1], int):
            t1 = hasil_and[case_temp[or_index[i]-1]]
        else:
            t1 = get_doc_id(case_temp[or_index[i]-1])
        
        if isinstance(case_temp[or_index[i]+1], int):
            t2 = hasil_and[case_temp[or_index[i]+1]]
        else:
            t2 = get_doc_id(case_temp[or_index[i]+1])
            
        hasil_temp = t1 + t2
        hasil +=hasil_temp
        
    return hasil

## Contoh Kasus

In [45]:
case2 = ["buyback", "OR", "chris", "AND", "get"]
case1 = ["concern", "OR", "~be", "AND", "~chris", "OR", "buyback", "AND", "china"]
case3 = ["concern", "OR", "~be", "AND", "~buyback", "OR", "china", "AND", "concern", "AND", "a"]
case4 = 'buyback chris AND get'

In [46]:
import PySimpleGUI as sg
sg.ChangeLookAndFeel('Dark')

layout = [
    [sg.Text('Kueri'),sg.Input('', key='search_kueri'), sg.Button('Cari')],
    [sg.Output(size=(100, 30))]
]

window = sg.Window('Search Engine', layout)

In [47]:
def get_atribut(doc_id):
    file_path = "DataRouter\\Doc" + str(doc_id) + ".txt"
    
    txt = readText(file_path)
    
    titlePattern = "<TITLE>(.+)</TITLE>"
    title = re.findall(titlePattern, txt)[0]
    
    datePattern = "<DATE>(.+)</DATE>"
    date = re.findall(datePattern, txt)[0]
    
    return title, date

# Main 

In [48]:
# main(case3)
while True:
    event, values = window.read()
    if event in ('Exit', None): break
    if event == 'Cari':
        a = values['search_kueri']
        query = queryProcessing(a)
        hasil = main(query)
        # hasil = list(dict.fromkeys(hasil))   
        for i in hasil:
            title, date = get_atribut(i)
            print(title,'(',date,')')
    print('=======================================================================================')  

window.close() 


TypeError: 'NoneType' object is not iterable